In [ ]:
import numpy as np
import pandas as pd
import torch
import torchvision
import cv2
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import sys
import copy
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
from torch.optim import lr_scheduler
from random import random
from tqdm.notebook import tqdm
from skimage import color
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
from torchvision import datasets, models, transforms
from sklearn.utils import shuffle
from kmeans_pytorch import kmeans, kmeans_predict

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth', None)
train_path=pd.read_csv('./MURA-v1.1/train_image_paths.csv', header=None)
train_path = shuffle(train_path)
train_path = train_path.reset_index()
train_path = train_path.drop('index',axis=1)
tmp=train_path[0].copy()
def generate_part(x):
    x=x.split('/')
    x=x[2]
    x=x.split('_')
    x=x[1]
    return x
tmp=tmp.apply(generate_part)
train_path['part']=tmp
train_path

In [ ]:
train_path['part'].unique()

In [ ]:
train_wrist_path = train_path[train_path.part == 'WRIST']#wrist
train_wrist_path=train_wrist_path.reset_index()
train_wrist_path.drop(['index','part'],axis=1,inplace=True)
train_wrist_path

In [ ]:
test_path=pd.read_csv('./MURA-v1.1/valid_image_paths.csv', header=None)
test_path = shuffle(test_path)
test_path = test_path.reset_index()
test_path = test_path.drop('index',axis=1)
tmp=test_path[0].copy()
tmp=tmp.apply(generate_part)
test_path['part']=tmp
test_path

In [ ]:
test_wrist_path = test_path[test_path.part == 'WRIST']
test_wrist_path=test_wrist_path.reset_index()
test_wrist_path.drop(['index','part'],axis=1,inplace=True)
test_wrist_path

In [ ]:
sep_path=train_wrist_path[0][1].split('/')
sep_path[-2][-8:]

In [ ]:
img=cv2.imread(train_wrist_path[0][0])
plt.imshow(img)

In [ ]:
img=color.rgb2gray(img)

In [ ]:
img=img*255
img

In [ ]:
img=img.astype('uint8')

In [ ]:
torch.cuda.empty_cache()

In [ ]:
img

In [ ]:
eq = cv2.equalizeHist(img)
plt.imshow(eq)

In [ ]:
unlabel_ratio=50
max_data_num=len(train_wrist_path)
labeled_index=int(max_data_num/unlabel_ratio)
labeled_df=train_wrist_path[:labeled_index]
unlabeled_df=train_wrist_path[labeled_index:]
unlabeled_df=unlabeled_df.reset_index()

In [ ]:
def generate_label(addr):
    addr=addr.split('/')
    if(addr[-2][-8:]=='positive'):
        return 1
    else:
        return 0
tmp=labeled_df[0]
tmp=tmp.apply(generate_label)
labeled_df[1]=tmp
labeled_data_label=np.array(tmp)

tmp=test_wrist_path[0]
tmp=tmp.apply(generate_label)
test_wrist_path[1]=tmp



In [ ]:
print(len(labeled_df))

In [ ]:
class xray_dataset(Dataset):#繼承Dataset物件
    def __init__(self,df,labeled):
        super().__init__()
        self.df=df
        self.labeled=labeled
    def __getitem__(self,index):
        #通常會在get item裡面實作data augmentation、transformation
        img=cv2.imread(self.df[0][index])
        img=color.rgb2gray(img)
        img=img*255
        img=img.astype('uint8')
        img=cv2.equalizeHist(img)
        if(img.shape[0]>img.shape[1]):#make image's long side parallel with x-axis
            if(random()>0.5):
                img=cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
            else:
                img=cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE)
        img = cv2.resize(img, (350, 200), interpolation=cv2.INTER_CUBIC)#width,height
        img=np.array([img]).astype('float32')#from 2 dim to 3 dim
#         img=cv2.resize(img, None,fx=0.4,fy=0.4, interpolation = cv2.INTER_AREA)

        label=0
        if(self.labeled==False):
            label=-1
        else:
            label=self.df[1][index]
        label=np.array(label).astype('float32')
        return img,label
        
    def __len__(self):
        return len(self.df)
labeled_dataset=xray_dataset(labeled_df,labeled=True)
unlabeled_dataset=xray_dataset(unlabeled_df,labeled=False)
train_dataset=xray_dataset(train_wrist_path,labeled=False)#used to train K-means
test_dataset=xray_dataset(test_wrist_path,labeled=True)

In [ ]:
plt.imshow(unlabeled_dataset[0][0][0])#[data index][img/label]

In [ ]:
unlabeled_dataset[0][0].shape

In [ ]:
class model(models.resnet.ResNet):
    def __init__(self,block,layers,num_classes):
        super(model, self).__init__(block,layers,num_classes)
        self.inplanes=64
        self.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.conv1.requires_grad=True

In [ ]:
model_res = model(block=models.resnet.Bottleneck,layers=[3, 4, 23, 3],num_classes=2)
model_res.fc = nn.Sequential(nn.Flatten(),                
                                nn.Linear(in_features=2048,out_features=1000),
                                nn.ReLU(inplace=True),
                                nn.Linear(in_features=1000,out_features=100),
                                nn.ReLU(inplace=True),
                                nn.Linear(in_features=100,out_features=2),
                                nn.Sigmoid()
        )
for param in model_res.parameters():
    param.requires_grad = True
model_res=model_res.float()
if torch.cuda.is_available():
    model_res=model_res.cuda()

In [ ]:
labeled_dataloader=DataLoader(labeled_dataset,batch_size=10,shuffle=False)
unlabeled_dataloader=DataLoader(unlabeled_dataset,batch_size=10,shuffle=False)
train_dataloader=DataLoader(train_dataset,batch_size=1,shuffle=False)
test_dataloader=DataLoader(test_dataset,batch_size=5,shuffle=True)

In [ ]:
def mixup_data(x,y,alpha=1.0):
    if alpha>0:
        lam=np.random.beta(alpha,alpha)
    else:
        lam=1
    batch_size=x.size()[0]
    if torch.cuda.is_available():
        index=torch.randperm(batch_size).cuda()
    else:
        index=torch.randperm(batch_size)
    
    mixed_x=lam*x+(1-lam)*x[index,:]
    y_a,y_b=y,y[index]
    return mixed_x,y_a,y_b,lam

In [ ]:
def mixed_criterion(criterion,pred,y_a,y_b,lam):
    return lam*criterion(pred,y_a)+(1-lam)*criterion(pred,y_b)

In [ ]:
def train(model_res,data_loader,test_dataloader,binary,epoch=20,learning_rate=1e-4,wd=3e-3,lr_sche_step=7,use_mixup=False):
    metrics_list={'train_loss':[],'test_loss':[]}
    criterionBCE=nn.BCELoss()
    criterionCE=nn.CrossEntropyLoss()
    optimizer=torch.optim.Adam(model_res.parameters(),lr=learning_rate,weight_decay=wd)
    scheduler=lr_scheduler.StepLR(optimizer,step_size=lr_sche_step,gamma=0.1)  
    best_f1=0.0
    best_model=copy.deepcopy(model_res.state_dict())
    total=len(data_loader.dataset)
    for i in range(epoch):
        print("epoch:",i)
        model_res.train()
        success=0
        loss_sum=0
        total_iteration=0.0
        for idx,(data,label) in enumerate(tqdm(data_loader)):   
            total_iteration+=1
            data.requires_grad_(True)
            if torch.cuda.is_available():
                data=data.cuda()
                label=label.cuda()
            
            if(use_mixup):
                data,label_a,label_b,lam=mixup_data(data,label)
                
            optimizer.zero_grad()
            output=model_res(data)
            if(binary):
                if(use_mixup):
                    loss=mixed_criterion(criterionBCE,output[:,0],label_a,label_b,lam)
                else:
                    loss=criterionBCE(output[:,0],label)
            else:
                if(use_mixup):
                    label_a=label_a.long()
                    label_b=label_b.long()
                    loss=mixed_criterion(criterionCE,output,label_a,label_b,lam)
                else:
                    label=label.long()
                    loss=criterionCE(output,label)
            loss.backward()
            loss.detach_()
            loss_sum+=loss
            optimizer.step()
            if(binary):
                preds = (output>0.5)
    #             print("preds",preds.type(torch.int)[:,0])
                success+=torch.sum(preds.type(torch.int)[:,0]==label.type(torch.int))
            else:
                _,preds=torch.max(output,1)
                label=label.long()
                success+=torch.sum(preds==label)
        print("training_accuracy:{:.3f}".format(float(success) / float(total)))
        metrics_list['train_loss'].append(loss_sum/total_iteration)
        cur_f1,test_loss=test(model_res,test_dataloader,binary)
        metrics_list['test_loss'].append(test_loss)
        if(cur_f1>best_f1):
            best_f1=cur_f1
            best_model=copy.deepcopy(model_res.state_dict())
        scheduler.step()
        
    return best_model,model_res,metrics_list

In [ ]:
@torch.no_grad()
def test(model_res,data_loader,binary):
    model_res.eval()
    criterionBCE=nn.BCELoss()
    criterionCE=nn.CrossEntropyLoss()
    total=len(data_loader.dataset)
    success=0
    loss_sum=0
    total_iteration=0.0
    prec_sum = 0.0
    recall_sum =0.0
    f1_sum = 0.0 
    for idx,(data,label) in enumerate(tqdm(data_loader)):
        total_iteration+=1
        if torch.cuda.is_available():
            data=data.cuda()
            label=label.cuda()
        output=model_res(data)
        if(binary):
            loss=criterionBCE(output[:,0],label)
        else:
            label=label.long()
            loss=criterionCE(output,label)
        loss_sum+=loss
        if(binary):
            preds = (output>0.5)
            success+=torch.sum(preds.type(torch.int)[:,0]==label.type(torch.int))
        else:
            _,preds=torch.max(output,1)
            success+=torch.sum(preds==label)
        y = np.array(label.cpu())
        pred = np.array(preds.cpu())
        prec_sum += metrics.precision_score(y, pred,average='macro',zero_division=1)
        recall_sum += metrics.recall_score(y, pred,average='macro',zero_division=1)
        f1_sum += metrics.f1_score(y, pred,average='macro',zero_division=1)
    acc=float(success) / float(total)
    print("testing_accuracy:{:.3f}".format(acc))
    print("testing_loss:{:.3f}".format(loss_sum/total_iteration))
    print("testing_precision:{:.3f}".format(prec_sum/total_iteration))
    print("testing_recall:{:.3f}".format(recall_sum/total_iteration))
    print("testing_f1:{:.3f}".format(f1_sum/total_iteration))
    return f1_sum/total_iteration,loss_sum/total_iteration

In [ ]:
epoch=40
model_best,model_res,metrics_list=train(model_res,labeled_dataloader,test_dataloader,binary=False,epoch=epoch,wd=3e-3,lr_sche_step=20,use_mixup=False)
plt.plot(np.arange(1,epoch+1,1), metrics_list['train_loss'], '-',color='blue',label="train")
plt.plot(np.arange(1,epoch+1,1), metrics_list['test_loss'], '-',color='orange',label="test")
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()

In [ ]:
model_res.load_state_dict(model_best)

In [ ]:
test(model_res,test_dataloader,binary=False)

In [ ]:
@torch.no_grad()
def generate_feature_vector(model,dataloader):
    vector_list=[]
    for (data,label) in tqdm(dataloader):
        if(torch.cuda.is_available()):
            data=data.cuda()
        output=model(data)
        output=output.cpu()
        output=np.array(output)
        vector_list.append(output[0,:,0,0])
    return vector_list

In [ ]:
def freeze_model_and_reset_fc(model,output_dim):
    for param in model.parameters():
        param.requires_grad = False
    model.fc= nn.Sequential(nn.Flatten(),                
                            nn.Linear(in_features=2048,out_features=1000),
                            nn.ReLU(inplace=True),
                            nn.Linear(in_features=1000,out_features=100),
                            nn.ReLU(inplace=True),
                            nn.Linear(in_features=100,out_features=output_dim),
                            nn.Sigmoid()
                            )
    model.fc.requires_grad=True
    return model


In [ ]:
class knn_dataset(Dataset):
    def __init__(self,df,label):
        super().__init__()
        self.df=df
        self.label=label
    def __getitem__(self,index):
        img=cv2.imread(self.df[0][index])
        img=color.rgb2gray(img)
        img=img*255
        img=img.astype('uint8')
        img=cv2.equalizeHist(img)
        if(img.shape[0]>img.shape[1]):#make image's long side parallel with x-axis
            if(random()>0.5):
                img=cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
            else:
                img=cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE)
        img = cv2.resize(img, (350, 200), interpolation=cv2.INTER_CUBIC)#width,height
        img=np.array([img]).astype('float32')#from 2 dim to 3 dim
        label=self.label[index]
        label=np.array(label).astype('float32')
        return img,label
        
    def __len__(self):
        return len(self.df)


In [ ]:
def first_cycle_training(model_res,cycle_num,epoch):
    for i in range(cycle_num):
        res_conv=nn.Sequential(*list(model_res.children())[:-1])#model_res without the FC layer
        feature_vector=generate_feature_vector(res_conv,train_dataloader)
        feature_vector_array=np.array(feature_vector)
        feature_vector_array=torch.from_numpy(feature_vector_array)
        model_res=freeze_model_and_reset_fc(model_res,output_dim=50)
        model_res=model_res.cuda()
        if torch.cuda.is_available():
            device = torch.device('cuda:0')
        else:
            device = torch.device('cpu')
            
        cluster_ids_x, cluster_centers = kmeans(
            X=feature_vector_array, num_clusters=10, distance='euclidean', device=device
        )
        data_len=len(train_wrist_path)
        data_index=int(data_len*0.8)
        knn_train_data=knn_dataset(train_wrist_path[:data_index],cluster_ids_x[:data_index])
        knn_test_data=knn_dataset(train_wrist_path[data_index:].reset_index(),cluster_ids_x[data_index:])
        knn_train_loader=DataLoader(knn_train_data,batch_size=10,shuffle=True)
        knn_test_loader=DataLoader(knn_test_data,batch_size=10,shuffle=True)

        model_best,model_res,metrics_list=train(model_res,knn_train_loader,knn_test_loader,binary=False,epoch=int(epoch/3),wd=0)
        plt.plot(np.arange(1,int(epoch/3)+1,1), metrics_list['train_loss'], '-',color='blue',label="train")
        plt.plot(np.arange(1,int(epoch/3)+1,1), metrics_list['test_loss'], '-',color='orange',label="test")
        plt.xlabel('epoch')
        plt.ylabel('loss')
        # plt.savefig(self.log_dir/'metrics.png')
        plt.show()
        
        for param in model_res.parameters():
            param.requires_grad = True
        model_best,model_res,metrics_list=train(model_res,knn_train_loader,knn_test_loader,binary=False,epoch=epoch,learning_rate=1e-4,wd=0,lr_sche_step=7)
        plt.plot(np.arange(1,epoch+1,1), metrics_list['train_loss'], '-',color='blue',label="train")
        plt.plot(np.arange(1,epoch+1,1), metrics_list['test_loss'], '-',color='orange',label="test")
        plt.xlabel('epoch')
        plt.ylabel('loss')
        # plt.savefig(self.log_dir/'metrics.png')
        plt.show()
    return model_best,model_res

In [ ]:
first_model_best,model_res=first_cycle_training(model_res,2,10)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
def second_cycle_training(model_res,cycle_num,epoch,test_dataloader):
    for i in range(cycle_num):
        res_conv=nn.Sequential(*list(model_res.children())[:-1])#model_res without the FC layer
        labeled_dataloader=DataLoader(labeled_dataset,batch_size=1,shuffle=False)
        labeled_feature_vector=generate_feature_vector(res_conv,labeled_dataloader)
        labeled_feature_vector_array=np.array(labeled_feature_vector)
        labeled_feature_vector_array=torch.from_numpy(labeled_feature_vector_array)
        
        unlabeled_dataloader=DataLoader(unlabeled_dataset,batch_size=1,shuffle=False)
        unlabeled_feature_vector=generate_feature_vector(res_conv,unlabeled_dataloader)
        unlabeled_feature_vector_array=np.array(unlabeled_feature_vector)
        unlabeled_feature_vector_array=torch.from_numpy(unlabeled_feature_vector_array)
        
        if torch.cuda.is_available():
            device = torch.device('cuda:0')
        else:
            device = torch.device('cpu')
        
        knn=KNeighborsClassifier(n_neighbors=5)
        knn.fit(labeled_feature_vector_array,labeled_data_label)
        pseudo_label=knn.predict(unlabeled_feature_vector_array)
        pseudo_labeled_df=train_wrist_path
        pseudo_labeled_df[1]=pd.DataFrame(np.concatenate([labeled_data_label,pseudo_label]))
        pseudo_labeled_dataset=xray_dataset(pseudo_labeled_df,labeled=True)
        pseudo_labeled_dataloader=DataLoader(pseudo_labeled_dataset,batch_size=10,shuffle=False)
        
        model_res=freeze_model_and_reset_fc(model_res,output_dim=2)
        model_res=model_res.cuda()
        
        data_len=len(train_wrist_path)
        data_index=int(data_len*0.8)

        model_best,model_res,metrics_list=train(model_res,pseudo_labeled_dataloader,test_dataloader,binary=False,epoch=int(epoch/3),wd=0)
        plt.plot(np.arange(1,int(epoch/3)+1,1), metrics_list['train_loss'], '-',color='blue',label="train")
        plt.plot(np.arange(1,int(epoch/3)+1,1), metrics_list['test_loss'], '-',color='orange',label="test")
        plt.xlabel('epoch')
        plt.ylabel('loss')
        # plt.savefig(self.log_dir/'metrics.png')
        plt.show()
        
        for param in model_res.parameters():
            param.requires_grad = True
        model_best,model_res,metrics_list=train(model_res,pseudo_labeled_dataloader,test_dataloader,binary=False,epoch=epoch,learning_rate=1e-4,wd=5e-3,lr_sche_step=7)
        plt.plot(np.arange(1,epoch+1,1), metrics_list['train_loss'], '-',color='blue',label="train")
        plt.plot(np.arange(1,epoch+1,1), metrics_list['test_loss'], '-',color='orange',label="test")
        plt.xlabel('epoch')
        plt.ylabel('loss')
        # plt.savefig(self.log_dir/'metrics.png')
        plt.show()
    return model_best,model_res
second_model_best,model_res=second_cycle_training(model_res,2,10,test_dataloader)

In [ ]:
model_res.load_state_dict(second_model_best)

In [ ]:
test(model_res,test_dataloader,binary=False)

In [ ]:
epoch=40
model_best,model_res,metrics_list=train(model_res,labeled_dataloader,test_dataloader,binary=False,epoch=epoch,wd=3e-3,lr_sche_step=20,use_mixup=False)
plt.plot(np.arange(1,epoch+1,1), metrics_list['train_loss'], '-',color='blue',label="train")
plt.plot(np.arange(1,epoch+1,1), metrics_list['test_loss'], '-',color='orange',label="test")
plt.xlabel('epoch')
plt.ylabel('loss')
# plt.savefig(self.log_dir/'metrics.png')
plt.show()

In [ ]:
model_res.load_state_dict(model_best)
test(model_res,test_dataloader,binary=False)

In [ ]:
model_mixup = model(block=models.resnet.Bottleneck,layers=[3, 4, 23, 3],num_classes=2)
model_mixup.fc = nn.Sequential(nn.Flatten(),                
                                nn.Linear(in_features=2048,out_features=1000),
                                nn.ReLU(inplace=True),
                                nn.Linear(in_features=1000,out_features=100),
                                nn.ReLU(inplace=True),
                                nn.Linear(in_features=100,out_features=2),
                                nn.Sigmoid()
        )
for param in model_mixup.parameters():
    param.requires_grad = True
model_mixup=model_mixup.float()
if torch.cuda.is_available():
    model_mixup=model_mixup.cuda()

In [ ]:
epoch=80
model_mixup_best,model_mixup,metrics_list=train(model_mixup,labeled_dataloader,test_dataloader,binary=False,epoch=epoch,wd=3e-3,lr_sche_step=25,use_mixup=False)
plt.plot(np.arange(1,epoch+1,1), metrics_list['train_loss'], '-',color='blue',label="train")
plt.plot(np.arange(1,epoch+1,1), metrics_list['test_loss'], '-',color='orange',label="test")
plt.xlabel('epoch')
plt.ylabel('loss')
# plt.savefig(self.log_dir/'metrics.png')
plt.show()

In [ ]:
model_mixup.load_state_dict(model_mixup_best)
test(model_mixup,test_dataloader,binary=False)